In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
import seaborn as sns 
import matplotlib.pyplot as plt 
import re 
from mpl_toolkits.basemap import Basemap 
import sklearn
%matplotlib inline

# preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
import pandas_profiling as pp

from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.metrics import mean_absolute_error,r2_score




In [ ]:
df = pd.read_csv('../input/craigslist-carstrucks-data/vehicles.csv')

In [ ]:
df.head()


In [ ]:
print(len(df.columns))
print(df.columns)
print(df.size)
print(df.shape)

# **Show Statistics and Preprocessing**

Plot the nulls in each column 

In [ ]:
nulls = df.isnull().sum().sort_values(ascending=False).div(len(df))
plt.figure(figsize=(24,12))
sns.barplot(x=nulls.index, y=nulls.values)
plt.title("Percent of missing data")
plt.show()

As seen above the county will be removed as it's all null
the size coulmn will be removed because the null percentage is about 64% also because the size of the car doesn't necessiraly effect it's price as the MINICOPPER is small-sized but high price as the 4*4 which is big-size,
But despite that the condition, vin and cylinders are all have many null values, we see that they have a lot of effect on the price because in egypt, if the car hasn't id verification number, it considered to be stolen. The number of cylinders show the power of the motor and the condition of the car for sure if it's new, it has higher price. 
The paint color column will be removed as it hasn't effect on the price.
Image_Url hasn't any effect as it's not related to the car
The description will be removed to make the process easier and because we for sure it's not accurate as the seller will speak good about the car. 


In [ ]:
data = df.drop(["county","paint_color","description","image_url","size"],axis=1)
print(data.size)
print(data.shape)
print(len(data.columns))
data.head()

As shown below , there are alot of columns are not numeric so we should convert it 

In [ ]:
data.info()

Adjust the cylinders column to make all the values integers as it's come in form number+"cylinders" and if null, put -1

In [ ]:
def cylinders_adjustment(row):
    if type(row["cylinders"]) is str:
        cyl = re.findall(r"(\d) cylinders", row["cylinders"])
        if len(cyl) != 0:
            return int(cyl[0])
        else:
            return -1
    else:
        return -1
        
#data["cylinders"] = data.apply(cylinders_adjustment,axis=1)

In [ ]:
def condition_adjustment(row):
    if type(row["condition"]) is str:
        if(row["condition"]=='excellent'):
            return 0
        elif(row["condition"]=='good' or row["condition"]=='fair'):
            return 1
        elif(row["condition"]=='new'or row["condition"]=='like new'):
            return 2
        else:
            return -1
    else:
        return -1
        

I just tried to remove the null rows from the data and found it +96k and as you said that the accuracy of the model is not the important thing so I will use remove the null but I have did some preprocessing above through the two functions
"condition_adjustment" and "cylinders_adjustment" so I will leave them and also I found an automatic way to convert the data from string to a numberical value using LabelEncoder method, I will use it below instead of implementing it. 

In [ ]:
data = data.dropna()
print(data.shape)
data.info()
data_copy= data.copy()
data_copy.shape

In [ ]:
manufacturers = data_copy["manufacturer"].value_counts().div(len(data_copy)).mul(100)
manufactuters_TOP20 = manufacturers[:20]

plt.figure(figsize=(16,8))
sns.barplot(x=manufactuters_TOP20.index, y=manufactuters_TOP20.values)
plt.title("20 most popular manufactureres in the USA")
plt.ylabel("Popularity in %")
plt.xticks(rotation=90)
plt.show()

In [ ]:
numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
categorical_columns = []
features = data.columns.values.tolist()
for col in features:
    if data[col].dtype in numerics: continue
    categorical_columns.append(col)
# Encoding categorical features
for col in categorical_columns:
    if col in data.columns:
        le = LabelEncoder()
        le.fit(list(data[col].astype(str).values))
        data[col] = le.transform(list(data[col].astype(str).values))

In [ ]:
data.info()

In [ ]:
data["year"] = data["year"].astype("int64")
data["odometer"] = data["odometer"].astype("int64")
data["lat"] = data["lat"].astype("int64")
data["long"] = data["long"].astype("int64")

In [ ]:
data.info()

In [ ]:
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
data.describe()

In [ ]:
pp.ProfileReport(data)

After the previous report , we notice that the price has 5373 values=0 and 1426 values=1 which are indication for corrupted data for sure so it must be removed. Also their are 2 cars with values (24500499$ and 1111111$ ) which is no possible for used car. 
So I will make condition to take only the data between 1000 and 375000

In [ ]:
data = data[data['price'] > 1000]
data = data[data['price'] < 375000]
data = data[data['model'] > 0]

In [ ]:
plt.figure(figsize=(20,10))

data_sub = data.sample(80000)

m = Basemap(projection='merc', # mercator projection
            llcrnrlat = 20,
            llcrnrlon = -170,
            urcrnrlat = 70,
            urcrnrlon = -60,
            resolution='l')

m.shadedrelief()
m.drawcoastlines() # drawing coaslines
m.drawcountries(linewidth=2) # drawing countries boundaries
m.drawstates(color='b') # drawing states boundaries
#m.fillcontinents(color='grey',lake_color='aqua')

for index, row in data_sub.iterrows():
    latitude = row['lat']
    longitude = row['long']
    x_coor, y_coor = m(longitude, latitude)
    m.plot(x_coor,y_coor,'.',markersize=0.9,c="red")

In [ ]:
plt.figure(figsize=(16,9))
sns.boxplot(x="year", y="price", data = data)
plt.title("Price of cars vs. manufacturing year")
plt.ylabel("Price [USD]")

max_year = data["year"].max()
min_year = data["year"].min()
steps = 2
lab = np.sort(data["year"].unique())[::2]
pos = np.arange(0,111,2)

plt.xticks(ticks=pos, labels=lab, rotation=90)
plt.show()

# Prepare to the model

In [ ]:
train_target = data["price"]
data_ = data.drop(columns=["price"])

In [ ]:
train0, test0, train_target0, test_target0  = train_test_split(data_, train_target, test_size=0.2, random_state=0)

In [ ]:
#For models from Sklearn
scaler = StandardScaler()
train0 = pd.DataFrame(scaler.fit_transform(train0), columns = train0.columns)

In [ ]:
train, test, target, target_test = train_test_split(train0, train_target0, random_state=0)

In [ ]:
print(train.info())
print(test.info())

In [ ]:
def acc_model(model,train,test):
    # Calculation of accuracy of model акщь Sklearn by different metrics   
    
    ytrain = model.predict(train)  
    ytest = model.predict(test)

    acc_train_r2_num = round(r2_score(target, ytrain) * 100, 2)
    print('acc(r2_score) for train =', acc_train_r2_num)   

    
    acc_test_r2_num = round(r2_score(target_test, ytest) * 100, 2)
    print('acc(r2_score) for test =', acc_test_r2_num)
    


In [ ]:
random_forest = RandomForestRegressor()
random_forest.fit(train, target)
acc_model(random_forest,train,test)